In [ ]:
DATA_PATH = "data"

In [8]:
# Imports
import pandas as pd
import numpy as np
import datetime 
from pathlib import Path
from tqdm.notebook import tqdm
import os

In [3]:

submission_df = pd.read_csv(os.path.join(DATA_PATH, "sample_submission.csv"))
test_user_ids = submission_df.Id

In [5]:
def replace_date(date_str):
    if len(date_str) < 8:
        return '2020-10-09'
    else:
        return date_str

def str2date(date_str):
    try:
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    except:
        date = datetime.datetime.strptime(date_str, '%d.%m.%Y')
    return date

def calculate_age(born):
    today = datetime.date.today()
    age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    if age < 15 or age > 60:
        age = 0
    return age

## Preprocess Users
user_df = pd.read_csv(os.path.join(DATA_PATH, "/kaggle_tab_1345/tab_4_user_profiles.csv"))
# Drop NaNs
user_df.dropna(inplace=True)
# # Fill NaNs
# user_df.bdate.fillna('2020-10-09', inplace=True)
user_df = user_df[user_df.user_id.isin(test_user_ids)]
# Replace incorrect dates
user_df.bdate = user_df.bdate.map(lambda x: replace_date(x))
# Convert all dates to datetime object
user_df.bdate = user_df.bdate.map(lambda x: str2date(x))
# Add new column with age
user_df["age"] = user_df.bdate.map(lambda x: calculate_age(x))
# Label code column
user_df.gender = user_df.gender.astype('category').cat.codes

user_df.drop_duplicates("user_id", inplace=True)
# users_df.head(3)

## Preprocess Citys
city_df = pd.read_csv(os.path.join(DATA_PATH, "tab_6_city.csv"))
city_df.city_name = city_df.city_name.astype('category').cat.codes
# city_df.head(3)

In [6]:
## Preprocess Orders
orders_df = pd.read_csv(os.path.join(DATA_PATH, "kaggle_tab_1345/tab_1_orders.csv"))
# Drop
orders_df.drop_duplicates(["order_id", "store_id"], inplace=True)
# Label code columns
orders_df.retailer = orders_df.retailer.astype('category').cat.codes
orders_df.platform = orders_df.platform.astype('category').cat.codes

# Merge city data
orders_df = orders_df.set_index('store_id').join(city_df.set_index('store_id'))

# Merge user data
orders_df = orders_df.set_index('user_id').join(user_df.set_index('user_id'))
orders_df.drop_duplicates("order_id", inplace=True)

In [9]:
pathes = sorted(list(Path(DATA_PATH).glob('*/tab_2*.csv')))
dfs = []
total_count = 0

for path in tqdm(pathes):
    # Read DF
    df = pd.read_csv(path)
#     print("Before:", len(df))
    
    # Filter and delete users not from test
    df = df[df.user_id.isin(test_user_ids)]

    # Drop column with long desciption
    df.drop(columns=["product_name"], inplace=True)
    
    # Label code column
    df.brand_name = df.brand_name.astype('category').cat.codes
    
    # Fill NaNs in one column
    df.master_category_id.fillna(-1, inplace=True)
    
    # Change data types to reduce memory footprint
    df.astype({
        'user_id': 'int32',
        'order_id': 'int32',
        'price': 'float32',
        'quantity': 'int32',
        'discount': 'float32',
        'product_id': 'int32',
        'brand_name': 'int32',
        'master_category_id': 'int32',
        'parent_category_id': 'int32'}, copy=False)
    total_count = total_count + len(df)
    dfs.append(df)

print("Total:", total_count)


Total: 21677070


In [10]:
products_df = pd.concat(dfs)

In [ ]:
# Can be slow, just wait =)
merged_df = products_df.set_index('order_id').join(orders_df.set_index('order_id'))
merged_df = merged_df.reset_index()
print("Total:", len(merged_df))

In [15]:
merged_df.head()

,user_id,line_item_id,price,quantity,discount,product_id,brand_name,master_category_id,parent_category_id,order_created_time,retailer,platform,city_name,gender,bdate,age
order_id,,,,,,,,,,,,,,,,
139128,1050,97633689,236.889999,1,0.0,5045454,3573,122.0,119,NaN,NaN,NaN,NaN,NaN,NaT,NaN
139128,1050,97633791,178.160004,1,0.0,73719,5154,122.0,119,NaN,NaN,NaN,NaN,NaN,NaT,NaN
139128,1050,97635201,173.279999,1,0.0,7000939,2846,85.0,84,NaN,NaN,NaN,NaN,NaN,NaT,NaN
139128,1050,97635312,73.690002,1,0.0,7386015,3342,85.0,84,NaN,NaN,NaN,NaN,NaN,NaT,NaN
139128,1050,97635414,282.140015,1,0.0,73291,2846,85.0,84,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [23]:
merged_df.fillna(value=-1, inplace=True)

In [24]:
merged_df.dtypes

order_id                int64
user_id                 int64
line_item_id            int64
price                 float64
quantity                int64
discount              float64
product_id              int64
brand_name              int16
master_category_id    float64
parent_category_id      int64
order_created_time     object
retailer              float64
platform              float64
city_name             float64
gender                float64
bdate                  object
age                   float64
dtype: object

In [25]:
# Change data types to reduce memory footprint
merged_df = merged_df.astype({
    'user_id': 'int32',
    'order_id': 'int32',
    'price': 'float32',
    'quantity': 'int32',
    'discount': 'float32',
    'product_id': 'int32',
    'brand_name': 'int32',
    'master_category_id': 'int32',
    'parent_category_id': 'int32',
    'retailer': 'int16',
    'platform': 'int8',
    'city_name': 'int16',
    'gender': 'int8',
    'age': 'int8',
    }, copy=True)

In [ ]:
merged_df.dtypes

In [26]:
merged_df.to_csv(os.path.join(DATA_PATH, "merged_data.csv"))